In [151]:
using Pkg
Pkg.activate(".")

  Activating project at `~/Desktop/Continuous MPS/MatrixProductBP.jl`


In [152]:
using Revise
using MatrixProductBP, MatrixProductBP.Models
using Graphs, IndexedGraphs, Statistics, Random, LinearAlgebra, TensorTrains, SparseArrays
import ProgressMeter; ProgressMeter.ijulia_behavior(:clear)
using TensorTrains: summary_compact
using SparseArrays
using JLD2
include("bp_fourier.jl");

In [153]:
seed = 4
rng = MersenneTwister(seed)

T = 4
# N = 3
# A = [0 1 1; 
#      1 0 0;
#      1 0 0;]
# N = 4
# A = [0 1 0 0; 
#      1 0 1 0; 
#      0 1 0 1; 
#      0 0 1 0]
# N = 7
# A = [0 1 0 0 0 0 0; 
#      1 0 1 0 0 0 0; 
#      0 1 0 1 0 0 0;
#      0 0 1 0 1 0 0;
#      0 0 0 1 0 1 0;
#      0 0 0 0 1 0 1;
#      0 0 0 0 0 1 0]
# N = 4
# A = [0 1 1 1;
#      1 0 0 1;
#      1 0 0 1;
#      1 1 1 0]
# !!! Here the calculations are wrong for edge 1->2 !!!
# N = 5
# A = [0 1 1 1 1; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0; 
#      1 0 0 0 0]
# N = 6
# A = [0 1 1 1 0 0; 
#      1 0 0 0 1 0; 
#      1 0 0 0 0 1; 
#      1 0 0 0 0 0; 
#      0 1 0 0 0 0; 
#      0 0 1 0 0 0]
# N = 7
# A = [0 1 1 1 1 0 1;
#      1 0 0 0 1 0 0; 
#      1 0 0 0 0 1 0; 
#      1 0 0 0 0 0 0; 
#      1 1 0 0 0 0 0; 
#      0 0 1 0 0 0 0; 
#      1 0 0 0 0 0 0]
# N = 8
# A = [0 1 1 1 0 0 1 0;
#      1 0 0 0 1 0 0 1; 
#      1 0 0 0 0 1 0 0; 
#      1 0 0 0 0 0 0 0; 
#      0 1 0 0 0 0 0 0; 
#      0 0 1 0 0 0 0 0; 
#      1 0 0 0 0 0 0 0;
#      0 1 0 0 0 0 0 0]

# g = IndexedBiDiGraph(A)

N = 20
c = 3
gg = erdos_renyi(N, c/N; seed)
g = IndexedBiDiGraph(gg)
A = zeros(Int,N,N)
for i in CartesianIndices(A)
     if !iszero(g.A[i])
         A[i] = 1
     end
end

β = 1.0
J = 1
h = 0.0
m⁰ = 1.0
K = 100

svd_trunc=TruncBond(4)

A

20×20 Matrix{Int64}:
 0  1  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1
 1  0  0  0  0  1  0  0  1  0  0  0  0  1  0  1  0  0  0  0
 0  0  0  0  1  0  0  1  0  1  0  0  0  0  1  1  0  0  0  0
 0  0  0  1  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
 0  0  1  0  0  0  0  0  1  1  0  1  0  0  1  0  0  0  1  0
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  1
 0  0  0  1  0  0  0  0  1  0  0  0  0  0  0  1  0  0  0  0
 0  0  1  0  0  1  0  1  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  0  1  0  1  0  0  0  0  1  1  0  0  0  0  0  0  0  1
 0  0  0  0  0  0  0  0  0  1  0  0  0  0  0  0  1  0  0  1
 0  0  0  0  0  1  0  0  0  1  0  0  0  1  0  1  0  0  1  1
 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  1  0  0  0  0
 0  0  1  0  0  0  0  0  0  0  0  1  0  0  1  0  0  0  0  1
 0  0  0  1  0  1  0  0  0  0  0  0  0  1  0  1  0  0  0  0
 0  0  1  1  0  0  0  1  1  0  0  1  1  0  1  0  0  1  0  0
 0  0  0  0  0  0  

In [154]:
w = [fill(IntegerGlauberFactor([J*A[j,i] for j in inneighbors(g,i)], h, β), T+1) for i in vertices(g)]
ϕᵢ = [t == 0 ? [(1-m⁰)/2, (1+m⁰)/2] : ones(2) for t in 0:T]
# ϕᵢ = [ones(2) for t in 0:T]
bp = mpbp(g, w, fill(2,N), T; ϕ=fill(ϕᵢ,N))

for μ in bp.μ
    for a in μ
        a.=rand.(rng)
        # a.=1.0
    end
end
# bp_fourier = deepcopy(bp)
iterate!(bp, maxiter=2; svd_trunc)

msg = deepcopy(bp.μ)
# jldsave("messages.jld2"; msg)

bp_fourier = convert_msg_beliefs(ComplexF64, bp)
# iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/100; svd_trunc)
;

Running MPBP: iter 2    Time: 0:00:00

In [155]:
# for (k,ed) in enumerate(edges(bp.g))
#     display(ed)
#     display(bp.μ[idx(ed)][3])
# end

In [156]:
iterate!(bp,maxiter=1; svd_trunc)

(1, CB_BP{ProgressUnknown, MatrixProductBP.var"#162#165"}(ProgressUnknown(false, false, 0, ProgressMeter.ProgressCore(:green, "Running MPBP: iter", 0.1, true, 0, VSCodeServer.IJuliaCore.IJuliaStdio{Base.PipeEndpoint, typeof(VSCodeServer.io_send_callback)}(IOContext(Base.PipeEndpoint(RawFD(25) open, 0 bytes waiting)), VSCodeServer.io_send_callback), false, 1, 1, ReentrantLock(nothing, 0x00000000, 0x00, Base.GenericCondition{Base.Threads.SpinLock}(Base.IntrusiveLinkedList{Task}(nothing, nothing), Base.Threads.SpinLock(0)), (2, 2, 2)), 0, 1, false, false, 1.737714194167711e9, 1.737714194167711e9, 1.737714194167711e9)), [[[2.0, 1.9839772254535382, 1.632896927200663, 1.6098911496327768, 1.3055606050739306], [2.0, 1.9973336799498662, 1.9963704770571558, 1.976160040483573, 1.985694398766464], [2.0, 1.9999677253006862, 1.9941904380306, 1.9805439968405405, 1.9588280203330632], [2.0, 1.9998406056118185, 1.942559415653844, 1.8844139268551343, 1.8888182482943447], [2.0, 1.861257625807304, 1.863613

In [157]:
iterate_fourier!(bp_fourier,K, maxiter=1, σ=1/50; svd_trunc)
;

In [158]:
import MatrixProductBP: marginalize
function marginalize(A::MPEM2{F}, var) where F
    if var==1
        return MPEM1{F}([@tullio b[m, n, xi] := a[m, n, xi, xj] for a in A]; z = A.z)
    elseif var==2
        return MPEM1{F}([@tullio b[m, n, xj] := a[m, n, xi, xj] for a in A]; z = A.z)
    end
end;

In [159]:
er_b = []
ptr = []
for i in vertices(bp.g)
    margs = marginals(bp.b[i])
    margs_fourier = marginals(bp_fourier.b[i])
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        ind = argmax(abs.(marg))
        push!(er_b, abs(marg_fourier[ind]-marg[ind]))
        push!(ptr, (i, t))
    end
end

display(maximum(er_b))
display(ptr[argmax(er_b)])

0.08261184265503685

(16, 2)

In [160]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ_marg = marginalize(bp.μ[idx(e)],1)
    margs = marginals(μ_marg)
    μ_fourier_marg = marginalize(bp_fourier.μ[idx(e)],1)
    margs_fourier = marginals(μ_fourier_marg)
    for t in 1:T+1
        marg = margs[t]
        marg_fourier = margs_fourier[t]
        push!(er_μ, abs(marg_fourier[1]-marg[1]))
        push!(ptr, (e, t))
    end
end

display(maximum(er_μ))
display(ptr[argmax(er_μ)])

0.003257831768131547

(Indexed Edge 18 => 2 with index 65, 5)

In [161]:
er_μ = []
ptr = []
for e in edges(bp.g)
    μ = bp.μ[idx(e)]
    μ_fourier = bp_fourier.μ[idx(e)]
    push!(er_μ, norm2m(μ, μ_fourier)/(norm(μ)*norm(μ_fourier)))
    push!(ptr, (e))
end

display(maximum(abs, er_μ))
display(ptr[argmax(abs.(er_μ))])

0.00016915347785817568

Indexed Edge 18 => 7 with index 66

In [162]:
bp_fourier.μ[1][3]

4×4×2×2 Array{ComplexF64, 4}:
[:, :, 1, 1] =
  -0.00525906-0.0737442im    …     0.136038-0.207691im
   0.00893021+0.105618im          -0.191024+0.300184im
 -0.000370751+2.644e-5im          0.0106558+0.00697956im
 -0.000165035-0.000504468im     -0.00597651+0.0171732im

[:, :, 2, 1] =
  -0.0197438-0.276854im     …    -0.109634+0.167379im
 -0.00238243-0.0281771im          -0.01094+0.0171916im
 -0.00137796+9.82692e-5im      -0.00610062-0.00399591im
  4.45713e-5+0.000136242im      -0.0002486+0.000714341im

[:, :, 1, 2] =
  -0.00819235-0.114876im     …   -0.0873841+0.133411im
    0.0139111+0.164527im           0.122705-0.192824im
 -0.000108839+7.76185e-6im       0.00844524+0.00553164im
  -4.84488e-5-0.000148095im     -0.00473674+0.0136108im

[:, :, 2, 2] =
   -0.0307557-0.431266im    …     0.0704311-0.107528im
   -0.0037112-0.0438925im         0.0070281-0.0110443im
 -0.000405321+2.89054e-5im        -0.004464-0.00292392im
   1.31104e-5+4.0075e-5im      -0.000181902+0.000522686im

In [163]:
node = 2
time = 3

ed = inedges(bp.g, node) |> first
@show ed
μ = bp.μ[idx(ed)]
μ_marg = marginalize(μ,1)
marg = marginals(μ_marg)[time]

ed_fourier = inedges(bp_fourier.g, node) |> first
@show ed_fourier
μ_fourier = bp_fourier.μ[idx(ed_fourier)]
μ_fourier_marg = marginalize(μ_fourier,1)
marg_fourier = marginals(μ_fourier_marg)[time]

@show marg
@show marg_fourier;

ed = Indexed Edge 1 => 2 with index 1
ed_fourier = Indexed Edge 1 => 2 with index 1
marg = [0.266784653551786, 0.7332153464482141]
marg_fourier = ComplexF64[0.2669103108188348 + 8.115099437314236e-16im, 0.7330896891811652 - 8.115099437314236e-16im]


In [164]:
node = 1
time = 3

edd = outedges(bp.g, node)
for ed in edd
    display(ed)
    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    marg = marginals(μ_marg)[time]

    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)
    marg_fourier = marginals(μ_fourier_marg)[time]

    display(marg)
    display(marg_fourier)
end

Indexed Edge 1 => 2 with index 1

2-element Vector{Float64}:
 0.266784653551786
 0.7332153464482141

2-element Vector{ComplexF64}:
 0.2669103108188348 + 8.115099437314236e-16im
 0.7330896891811652 - 8.115099437314236e-16im

Indexed Edge 1 => 3 with index 2

2-element Vector{Float64}:
 0.26001636627695407
 0.7399836337230459

2-element Vector{ComplexF64}:
 0.26013913544220973 - 5.916666604347428e-17im
  0.7398608645577903 + 5.916666604347426e-17im

In [165]:
for ed in edges(bp.g)
    display(ed)

    μ = bp.μ[idx(ed)]
    μ_marg = marginalize(μ,1)
    μ_fourier = bp_fourier.μ[idx(ed)]
    μ_fourier_marg = marginalize(μ_fourier,1)

    for time in 1:T+1
        marg = marginals(μ_marg)[time]
        marg_fourier = marginals(μ_fourier_marg)[time]
        display("time = $time")
        display(marg)
        display(marg_fourier)
    end
end

Indexed Edge 1 => 2 with index 1

"time = 1"

2-element Vector{Float64}:
 0.0
 1.0

2-element Vector{ComplexF64}:
 1.1311556129102756e-16 - 6.374926730794041e-17im
     0.9999999999999998 + 6.374926730794051e-17im

"time = 2"

...

2-element Vector{Float64}:
 7.13215390207186e-5
 0.9999286784609792

2-element Vector{ComplexF64}:
 0.0002802951433819473 - 3.7898251996463604e-16im
    0.9997197048566181 + 3.7898251996463604e-16im

"time = 5"

2-element Vector{Float64}:
 -0.00030156394853032625
  1.0003015639485304

2-element Vector{ComplexF64}:
 -0.00019792176131370253 + 1.2801414697766628e-16im
      1.0001979217613137 - 1.2801414697766623e-16im